# Data Collection

Multiple sources were used for data collection. For daily features and prices we opted to gather the data directly from blockchain.info through their flexible API. 
In order to obtain higher-frequency data, several Node.js scripts were deployed onto an Amazon EC2 instance. The scripts were continuously downloading ticker and limit order book data from the Bitstamp exchange through the exchange's Websocket API and stored in an Amazon DynamoDB for future access. Since we could not reconstruct the limit order book accurately, we opted to use Bitstamp limit order book data from CryptoAPI. This consisted the top 20 bid-ask prices and volumes with 10 seconds frequencies. 

# References

* http://www.sciencedirect.com/science/article/pii/S0925231203003722
* http://ac.els-cdn.com/S0957417400000270/1-s2.0-S0957417400000270-main.pdf?_tid=3a06fc62-1d5b-11e6-877f-00000aab0f27&acdnat=1463619013_cba9f7ee840313639128ce15571f73ac
* Technical Analysis of Stock Trends, Robert D. Edwards and John Magee
* https://www.jbs.cam.ac.uk/fileadmin/user_upload/research/workingpapers/wp0030.pdf
* http://www.sciencedirect.com/science/article/pii/0261560692900483

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information deap, matplotlib, numpy, pandas, seaborn, sklearn

Software versions
Python 3.5.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.1.2
OS Linux 3.13.0 86 generic x86_64 with debian jessie sid
deap 1.0
matplotlib 1.5.1
numpy 1.10.4
pandas 0.18.0
seaborn 0.7.0
sklearn 0.17.1
Wed May 25 12:44:25 2016 BST

In [2]:
%matplotlib inline

from IPython.display import display
from ipywidgets import widgets
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict as od
import random
import re
import seaborn as sns

#pd.set_option('html', False)
np.set_printoptions(threshold=np.nan)
sns.set()

# Transform collected trade data into proper format and export it

In [137]:
path = '../btc-data/BTC_Trades_raw.csv'
date_parse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

data = pd.read_csv(path)

data.TradeData = data.TradeData.map(lambda x: float(re.sub('[^0-9,.]', '', x)))
data.TradeID = data.TradeID.map(lambda x: float(re.sub('[^0-9,.]', '', x)))

data.rename(columns = {data.columns[0]: 'Amount', data.columns[1]: 'Price'}, inplace = True)
data.insert(0, 'Trade ID', data.index)

data['Date'] = data['Date'].map(date_parse)

data.set_index('Date', inplace = True)
data.sort_values(by = 'Trade ID', inplace = True)

data.to_csv(path_or_buf='../btc-data/BTC_Trades_clean.csv')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


# Transform collected order book data into proper format

In [15]:
path = '../btc-data/BTC_OB_raw.csv'
date_parse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

data = pd.read_csv(path, sep='",')

data.rename(columns = {data.columns[0]: 'LOB Data', 
                       data.columns[1]: 'ID', 
                       data.columns[2]: 'Date'}, inplace = True)

data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('asks', 'asks ', x))
data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('bids', 'bids ', x))
data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('[^0-9,.,asks ,bids ,]', '', x))
data['ID'] = data['ID'].map(lambda x: re.sub('\D', '', x))
data['Date'] = data['Date'].map(lambda x: re.sub('[^0-9,/,:, ]', '', x))

data['Date'] = data['Date'].map(date_parse)

bids = data['LOB Data'].map(lambda x: re.split(',', re.sub('bids ', '', re.sub('^asks [0-9,.]*', '', x))))
asks = data['LOB Data'].map(lambda x: re.split(',', re.sub('asks ', '', re.sub('bids [0-9,.]*', '', x)[:-1])))

data.insert(1, 'Bids', bids)
data.insert(1, 'Asks', asks)

data['Bids'] = data['Bids'].map(lambda x: list(zip(x[::2], x[1::2])))
data['Asks'] = data['Asks'].map(lambda x: list(zip(x[::2], x[1::2])))

data.set_index('Date', inplace = True)
data.drop('LOB Data', axis = 1, inplace = True)

data = data[['ID','Asks', 'Bids']]
data['ID'] = data['ID'].map(lambda x: int(x))
data['Asks'] = data['Asks'].map(lambda x: list(map(lambda t: (float(t[0]), float(t[1])), x)))
data['Bids'] = data['Bids'].map(lambda x: list(map(lambda t: (float(t[0]), float(t[1])), x)))
data.sort_values(by = 'ID', inplace = True)

#data.to_csv(path_or_buf='../btc-data/BTC_OB_clean_test.csv')

/home/dl3913/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


# Recreate the order book - NOT WORKING

In [129]:
askside = [ (373.1,0.53671),
            (373.23,17.24),
            (373.27,7.9555),
            (373.28,1.3363),
            (373.29,0.11771),
            (373.5,55.04631),
            (373.55,10.6796),
            (373.56,1.43616041),
            (373.59,0.26566839),
            (373.6,1.43882788),
            (373.61,1.43841224),
            (373.65,0.2630484),
            (373.71,1.69908455),
            (373.73,0.43790512),
            (373.75,1.54945824),
            (373.77,1.85661036),
            (373.78,0.17786362),
            (373.8,2.76357342),
            (373.81,1.73747649),
            (373.82,1.16956076)
]
bidside = [(372.85,2.61805646),
           (372.84,8.9237),
           (372.67,8.0285),
           (372.66,3.28509036),
           (372.6,0.02326771),
           (372.25,10.6798),
           (372.24,6.493),
           (372.2,2.0),
           (371.96,6.692),
           (371.74,0.04),
           (371.73,7.265),
           (371.7,0.02231636),
           (371.49,6.928),
           (371.4,0.02233438),
           (371.28,7.441),
           (371.1,0.02235244),
           (371.04,6.84),
           (370.9,0.262),
           (370.88,2.16480323),
           (370.86,45.75)
]


#print(data.tail())
#data.sort_values(by = 'ID', inplace = True)
#print(data.ix[5, 'Asks'])
    
for i in range(int(len(data)/100)):
    #print('Asks: ', askside)
    #print('To update: ', asks)
    
    for ask in data.ix[i, 'Asks']:
        found = False
        for j in range(len(askside)):
            price = askside[j][0]
            vol = askside[j][1]
        if not(found):
            if price == ask[0]:
                if ask[1] == 0:
                    del askside[j]
                else:
                    del askside[j]
                    askside.insert(j, ask)
                found = True
            elif price < ask[0]:
                if ask[1] > 0:
                    askside.insert(j, ask)
                found = True

    #while len(askside) > 20:
        #askside.remove(askside[-1])
            
    for bid in data.ix[i, 'Bids']:
        found = False
        for j in range(len(bidside)):
            price = bidside[j][0]
            vol = bidside[j][1]
        if not(found):
            if price == bid[0]:
                if bid[1] == 0:
                    del bidside[j]
                else:
                    del bidside[j]
                    bidside.insert(j, bid)
                found = True
            elif price > bid[0]:
                if bid[1] > 0:
                    bidside.insert(j, bid)
                found = True

    #while len(bidside) > 20:
        #bidside.remove(bidside[-1])

[(570.0, 2.23492593), (376.32, 1.93660647), (376.34, 2.67382177), (376.4, 0.34587516), (374.89, 3.66), (376.4, 0.64543474), (376.4, 0.34587516), (376.39, 0.40600253), (376.26, 23.29955958), (376.39, 0.10644295), (376.26, 23.0), (376.27, 0.71587967), (500.0, 322.33761047), (500.0, 322.64552102), (376.47, 0.74308352), (376.27, 0.41632009), (376.3, 0.29955958), (379.27, 1.96007331), (376.43, 8.51701224), (374.5, 1.27976122)]
16786
[(372.85, 2.61805646), (372.84, 8.9237), (372.67, 8.0285), (372.66, 3.28509036), (372.6, 0.02326771), (372.25, 10.6798), (372.24, 6.493), (372.2, 2.0), (371.96, 6.692), (371.74, 0.04), (371.73, 7.265), (371.7, 0.02231636), (371.49, 6.928), (371.4, 0.02233438), (371.28, 7.441), (371.1, 0.02235244), (371.04, 6.84), (370.9, 0.262), (370.88, 2.16480323), (370.51, 51.0)]
3523


In [80]:
askside = {373.1: 0.53671, 
           373.23: 17.24,
           373.27: 7.9555,
           373.28: 1.3363,
           373.29: 0.11771, 
           373.5: 55.04631,
           373.55: 10.6796,
           373.56: 1.43616041, 
           373.59: 0.26566839, 
           373.6: 1.43882788,
           373.61: 1.43841224,
           373.65: 0.2630484, 
           373.71: 1.69908455, 
           373.73: 0.43790512, 
           373.27: 7.9555, 
           373.55: 10.6796, 
           373.22: 1.3363, 373.49: 7.9934, 373.72: 10.6566, 373.28: 1.3363}

asks = {373.27: 7.9929, 373.28: 0.0, 373.49: 0.0, 373.55: 10.6849, 373.72: 0.0}

for ask in asks:
    #print(asks.get(ask))
    if ask in askside:
        if asks.get(ask) == 0:
            askside.pop(ask)
        else:
            askside.pop(ask)
            askside.update({ask: asks.get(ask)})
    elif asks.get(ask) > 0:
        askside.update({ask: asks.get(ask)})
        if len(askside) > 20:
            askside.pop(max(askside))

            
pair = (1,2)
bidside = [(372.85,2.61805646),
           (372.84,8.9237),
           (372.67,8.0285),
           (372.66,3.28509036),
           (372.6,0.02326771),
           (372.25,10.6798),
           (372.24,6.493),
           (372.2,2.0),
           (371.96,6.692),
           (371.74,0.04),
           (371.73,7.265),
           (371.7,0.02231636),
           (371.49,6.928),
           (371.4,0.02233438),
           (371.28,7.441),
           (371.1,0.02235244),
           (371.04,6.84),
           (370.9,0.262),
           (370.88,2.16480323),
           (370.86,45.75)
]
bidside = dict(bidside)
print(bidside)

{371.1: 0.02235244, 371.49: 6.928, 370.86: 45.75, 371.7: 0.02231636, 372.2: 2.0, 371.73: 7.265, 370.88: 2.16480323, 372.25: 10.6798, 372.85: 2.61805646, 371.28: 7.441, 371.4: 0.02233438, 370.9: 0.262, 371.74: 0.04, 372.67: 8.0285, 371.96: 6.692, 372.24: 6.493, 372.66: 3.28509036, 372.84: 8.9237, 371.04: 6.84, 372.6: 0.02326771}


# Collect and transform order book states from CryptoIQ

In [3]:
URL = 'https://cryptoiq.io/api/marketdata/orderbooktop/bitstamp/btcusd/2016-%s'

dates = pd.date_range(start = '1/1/2016', end = '5/1/2016', freq='H')

lob_data = pd.DataFrame()

for date in dates:
    time = str(date.month) + '-' + str(date.day) + '/' + str(date.hour)
    data = pd.read_json(URL % time)
    lob_data = lob_data.append(data)

#lob_data.to_csv(path_or_buf='../btc-data/BTC_LOB_collected.csv')

KeyboardInterrupt: 

# Convert indices to datetime format (rows: 939612)

In [5]:
lob_data.set_index('time', inplace=True)
lob_data.index = pd.to_datetime(lob_data.index)

# Create indices for evenly spaced time series

In [16]:
dates = pd.date_range(start = '1/1/2016 00:00:00', end = '5/1/2016 00:59:50', freq='10s')
#lob_data.to_csv(path_or_buf='../btc-data/BTC_LOB_collected.csv')

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 00:00:10',
               '2016-01-01 00:00:20', '2016-01-01 00:00:30',
               '2016-01-01 00:00:40', '2016-01-01 00:00:50',
               '2016-01-01 00:01:00', '2016-01-01 00:01:10',
               '2016-01-01 00:01:20', '2016-01-01 00:01:30',
               ...
               '2016-05-01 00:58:20', '2016-05-01 00:58:30',
               '2016-05-01 00:58:40', '2016-05-01 00:58:50',
               '2016-05-01 00:59:00', '2016-05-01 00:59:10',
               '2016-05-01 00:59:20', '2016-05-01 00:59:30',
               '2016-05-01 00:59:40', '2016-05-01 00:59:50'],
              dtype='datetime64[ns]', length=1045800, freq='10S')

In [7]:
lob_data['asks'] = lob_data['asks'].map(dict)
lob_data['bids'] = lob_data['bids'].map(dict)

# Re-index the LOB table with the evenly spaced time series, fill missing values with the nearest available prices (rows: 1045800) 

In [19]:
lob_data = lob_data.reindex(dates, method = 'nearest')

In [21]:
lob_data.to_csv(path_or_buf='../btc-data/BTC_LOB_collected.csv')